# `INSTRUCTIONS FOR IMAGE SPACE SETUP`

### Step 1 - Download 80k images:
1. Open file download_images.ipynb and run the code from top to bottom. 80,000+ images will begin to download in the 95k_Images folder.
Use ls -1 | wc -l in 95k_Images to view download progress.

### Step 2 - Create a folder subset of 1,000 Images:
1. /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
2. brew install coreutils
3. Run this on 95k_Images to get a subset: find 95k_Images/ -type f -print0 | shuf -n 1000 -z | xargs -0 -I{} cp {} subset_images/

### Step 3 - Download Image Space repo:
1. run this on 8_ImageSpace Folder: `git clone https://github.com/nasa-jpl-memex/image_space.git`

### Step 4 - Set environment variables:
1. `export IMAGE_SPACE=/Users/toddgavin/Desktop/USC_Classes/DSCI550-Data_Science_at_Scale/Assignment_3/GitHub/DSCI550_PixstoryDatasetAnalysisVisualizations/8_ImageSpace/image_space`
    - verify path: `echo $IMAGE_SPACE`
2. `export IMAGE_DIR=/Users/toddgavin/Desktop/USC_Classes/DSCI550-Data_Science_at_Scale/Assignment_3/GitHub/DSCI550_PixstoryDatasetAnalysisVisualizations/8_ImageSpace/test_images`
    - verify path: `echo $IMAGE_DIR`

### Step 5 - Build your Neural Net (AlexNet) based features for the images.
1. `cd "$IMAGE_SPACE/imagespace_smqtk" && ./smqtk_services.run_images.sh --docker-network deploy_imagespace-network --images "$IMAGE_DIR"`
    - If having trouble, try to tear down the old network: `docker network prune`
    - you should see `docker ps -a` showing NOTHING
    - `docker network ls --no-trunc` showing NOTHING

### Step 6 - Run Neural Net
1. Run neural net command: `cd $IMAGE_SPACE/scripts/deploy && IMAGE_DIR=$IMAGE_DIR docker-compose up -d`
    - To run on M1 mac, refer to next markdown cell.

### Step 7 - Seeing if containers are working
1. go to this site: http://localhost:8088/opsui and you should see the Apache OODT instance
2. go to this site: http://localhost:64622/solr/#/imagecatdev/query and you should see the Apache Solr instance
    - Need to check what number is associted with port 8081 on the imageCat docker container and change the `64622` number in the link to the matching port 8081 number


# `TRY CHANGING TO platform: linux/arm64/v8`

## `Run on M1 Chip`

#### To run on amd64 chip (M1 chip), need to add `platform` in docker-compose.yml (image_space/scripts/deploy) file to `amd64` such that the file under each docker container looks like:
```sh
version: "2"
services:
  imagespace-mongo:
    platform: linux/amd64
    image: mongo:3.0
    command: "--smallfiles"
    networks:
      - imagespace-network
  imagespace-solr:
    platform: linux/amd64
    image: solr:6.0
    networks:
      - imagespace-network
    entrypoint:
      - docker-entrypoint.sh
      - solr-precreate
      - imagespace
```

#### Change mongoDB docker version to support M1 chip
- https://hub.docker.com/_/mongo/tags
- `docker pull --platform linux/amd64 mongo:5.0.16`
- Then go into the docker-compose.yml file and change version to `image: mongo:5.0.16`